# Online Adaptive Forecasting

In [ ]:
%load_ext autoreload
%autoreload 2

import re
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import trajdata.visualization.vis as trajdata_vis

from torch.utils import data
from tqdm.notebook import tqdm
from trajectron.model.model_registrar import ModelRegistrar
from trajectron.model.model_utils import UpdateMode
from trajectron.model.trajectron import Trajectron
from collections import defaultdict
from pathlib import Path
from typing import DefaultDict, Dict, Final, List, Optional, Union
from trajdata import UnifiedDataset, AgentType, AgentBatch

# Set random seed for reproducibility across numpy, torch, and CUDA
seed = 0
np.random.seed(seed)  # Set numpy random seed
torch.manual_seed(seed)  # Set PyTorch random seed for CPU
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)  # Set PyTorch random seed for all CUDA devices

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Change this to suit your computing environment and folder structure!
# Set up data directories and cache location for the trajectory prediction models

# Directory where trajdata will cache processed data for faster loading
TRAJDATA_CACHE_DIR: Final[str] = "/home/iot-lab/.unified_data_cache"

# Directory containing the raw nuScenes dataset files
LYFT_SAMPLE_RAW_DATA_DIR: Final[str] = "/home/iot-lab/datasets/nuScenes"

# Specify which dataset sample to use (nuScenes mini version for faster experimentation)
sample_type = "nusc_mini"

In [ ]:
# Model paths for different trajectory prediction models
base_model = "models/nusc_mm_base_tpp-11_Sep_2022_19_15_45"  # Base model without adaptation
k0_model = "models/nusc_mm_k0_tpp-12_Sep_2022_00_40_16"      # K0 baseline model
adaptive_model = "models/nusc_mm_sec4_tpp-13_Sep_2022_11_06_01"  # Our adaptive model
oracle_model = "models/lyft_mm_base_tpp-11_Sep_2022_18_56_49"    # Oracle model for comparison

# Checkpoint epochs to load for each model
base_checkpoint = 20      # Epoch to load for base model
k0_checkpoint = 20        # Epoch to load for K0 model
adaptive_checkpoint = 20  # Epoch to load for adaptive model
oracle_checkpoint = 1     # Epoch to load for oracle model

# Dataset configuration
eval_data = "nusc_mini-mini_val"  # nuScenes mini validation dataset

# Time horizon configuration for trajectory prediction
history_sec = 2.0      # Seconds of historical trajectory data to use as input
prediction_sec = 6.0   # Seconds into the future to predict trajectories

In [ ]:
# Color palette definitions for visualization and plotting

# Colors for horizontal axis lines in plots
# Used for marking reference performance levels of different baseline models
AXHLINE_COLORS = {
    "Base": "#DD9787",      # Coral/salmon color for base model performance line
    "K0": "#A6C48A",        # Light green color for K0 baseline model line
    "Oracle": "#A2999E"     # Gray-purple color for oracle model performance line
}

# Main color palette for different models/methods in plots
# Used for plotting trajectories, performance curves, and other visualizations
SEABORN_PALETTE = {
    "Finetune": "#AA7C85",      # Muted pink for standard finetuning approach
    "K0": "#A6C48A",            # Light green for K0 baseline method
    "Ours+Finetune": "#2D93AD", # Teal blue for our adaptive method with finetuning
    "Ours": "#9F9FED",          # Light purple for our main adaptive method
    "Base": "#DD9787",          # Coral/salmon for base model without adaptation
    "K0+Finetune": "#67934D",   # Dark green for K0 method with finetuning
    "Oracle": "#A2999E"         # Gray-purple for oracle (ground truth) model
}

In [ ]:
def load_model(model_dir: str, device: str, epoch: int = 10, custom_hyperparams: Optional[Dict] = None):
    """
    Load a pre-trained Trajectron model from disk.
    
    Args:
        model_dir: Directory containing the saved model files
        device: Device to load the model on ('cpu' or 'cuda')
        epoch: Which training epoch checkpoint to load
        custom_hyperparams: Optional dictionary to override default hyperparameters
    
    Returns:
        tuple: (trajectron_model, hyperparams_dict)
    """
    # Construct path to the specific epoch checkpoint file
    save_path = Path(model_dir) / f'model_registrar-{epoch}.pt'

    # Initialize model registrar to manage model components
    model_registrar = ModelRegistrar(model_dir, device)
    
    # Load the original training configuration/hyperparameters from JSON
    with open(os.path.join(model_dir, 'config.json'), 'r') as config_json:
        hyperparams = json.load(config_json)
        
    # Override any hyperparameters with custom values if provided
    if custom_hyperparams is not None:
        hyperparams.update(custom_hyperparams)

    # Create the Trajectron model instance with loaded hyperparameters
    trajectron = Trajectron(model_registrar, hyperparams, None, device)
    
    # Set up the model's environment and training parameters
    trajectron.set_environment()
    trajectron.set_annealing_params()

    # Load the saved model weights from the checkpoint file
    checkpoint = torch.load(save_path, map_location=device)
    trajectron.load_state_dict(checkpoint["model_state_dict"], strict=False)

    return trajectron, hyperparams

In [ ]:
# Check if CUDA (GPU) is available on the system
if torch.cuda.is_available():
    # If GPU is available, set device to use the first GPU (cuda:0)
    device = 'cuda:0'
    # Set the current CUDA device to GPU 0 for PyTorch operations
    torch.cuda.set_device(0)
else:
    # If no GPU is available, fall back to using CPU
    device = 'cpu'
    
# Display the selected device (either 'cuda:0' or 'cpu')
device

'cuda:0'

In [ ]:
# Load the adaptive trajectory prediction model from disk
adaptive_trajectron, hyperparams = load_model(
    adaptive_model, device, epoch=adaptive_checkpoint,
    custom_hyperparams={"trajdata_cache_dir": TRAJDATA_CACHE_DIR,
                        "single_mode_multi_sample": True}
)

# Configure attention radius (interaction distance) between different agent types
# This defines how far agents look to consider other agents in their predictions
attention_radius = defaultdict(lambda: 20.0) # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 10.0  # Pedestrian-pedestrian interactions: 10m
attention_radius[(AgentType.PEDESTRIAN, AgentType.VEHICLE)] = 20.0     # Pedestrian-vehicle interactions: 20m  
attention_radius[(AgentType.VEHICLE, AgentType.PEDESTRIAN)] = 20.0     # Vehicle-pedestrian interactions: 20m
attention_radius[(AgentType.VEHICLE, AgentType.VEHICLE)] = 30.0        # Vehicle-vehicle interactions: 30m

# Configure raster map parameters for spatial context encoding
# These parameters control how the map is converted to a pixel grid
map_params = {"px_per_m": 2, "map_size_px": 100, "offset_frac_xy": (-0.75, 0.0)}

# Create dataset for online evaluation (agent-by-agent sequential processing)
# This dataset provides variable history lengths to simulate real-time inference
online_eval_dataset = UnifiedDataset(
    desired_data=[eval_data],                          # Use nuScenes mini validation data
    # desired_dt=0.5,                                  # Optional: downsample to 0.5s intervals
    history_sec=(0.1, history_sec),                    # Variable history: 0.1s to 2.0s
    future_sec=(prediction_sec, prediction_sec),       # Fixed future horizon: 6.0s
    agent_interaction_distances=attention_radius,      # Use configured attention radii
    incl_robot_future=hyperparams['incl_robot_node'],  # Include ego vehicle future if specified
    incl_raster_map=hyperparams['map_encoding'],       # Include map encoding if specified
    raster_map_params=map_params,                      # Map rasterization parameters
    only_predict=[AgentType.VEHICLE],                  # Only predict trajectories for vehicles
    no_types=[AgentType.UNKNOWN],                      # Exclude unknown agent types
    num_workers=0,                                     # Single-threaded data loading
    cache_location=TRAJDATA_CACHE_DIR,                 # Cache directory for processed data
    data_dirs={
        sample_type: LYFT_SAMPLE_RAW_DATA_DIR,         # Map dataset type to raw data directory
    },
    verbose=True
)

# Create dataset for batch evaluation (standard evaluation with fixed history)
# This dataset uses fixed history length for consistent batch processing
batch_eval_dataset = UnifiedDataset(
    desired_data=[eval_data],                          # Use same validation data
    # desired_dt=0.5,                                  # Optional: downsample to 0.5s intervals
    history_sec=(history_sec, history_sec),            # Fixed history: 2.0s to 2.0s
    future_sec=(prediction_sec, prediction_sec),       # Fixed future horizon: 6.0s
    agent_interaction_distances=attention_radius,      # Use same attention radii
    incl_robot_future=hyperparams['incl_robot_node'],  # Include ego vehicle future if specified
    incl_raster_map=hyperparams['map_encoding'],       # Include map encoding if specified
    raster_map_params=map_params,                      # Same map parameters
    only_predict=[AgentType.VEHICLE],                  # Only predict trajectories for vehicles
    no_types=[AgentType.UNKNOWN],                      # Exclude unknown agent types
    num_workers=0,                                     # Single-threaded data loading
    cache_location=TRAJDATA_CACHE_DIR,                 # Same cache directory
    data_dirs={
        sample_type: LYFT_SAMPLE_RAW_DATA_DIR,         # Same data directory mapping
    },
    verbose=True
)

Loading data for matched scene tags: ['mini_val-nusc_mini-boston', 'singapore-mini_val-nusc_mini']


Calculating Agent Data (Serially): 100%|██████████| 2/2 [00:00<00:00, 7073.03it/s]


2 scenes in the scene index.


Structuring Agent Data Index: 100%|██████████| 2/2 [00:00<00:00, 4324.02it/s]

Loading data for matched scene tags: ['mini_val-nusc_mini-boston', 'singapore-mini_val-nusc_mini']



Calculating Agent Data (Serially): 100%|██████████| 2/2 [00:00<00:00, 9167.88it/s]


2 scenes in the scene index.


Structuring Agent Data Index: 100%|██████████| 2/2 [00:00<00:00, 2844.56it/s]


In [ ]:
def get_dataloader(
    eval_dataset: UnifiedDataset,
    batch_size: int = 128,
    num_workers: int = 0,
    shuffle: bool = False
):
    """
    Create a PyTorch DataLoader for trajectory prediction evaluation.
    
    Args:
        eval_dataset: UnifiedDataset containing trajectory data
        batch_size: Number of samples per batch (default: 128)
        num_workers: Number of subprocesses for data loading (default: 0 for single-threaded)
        shuffle: Whether to shuffle data between epochs (default: False for evaluation)
    
    Returns:
        DataLoader: Configured PyTorch DataLoader for the dataset
    """
    return data.DataLoader(
        eval_dataset,
        # Use dataset's collate function to properly batch variable-length sequences
        # "right" padding ensures sequences are aligned for batch processing
        collate_fn=eval_dataset.get_collate_fn(pad_format="right"),
        # Enable memory pinning for faster GPU transfers, disable for CPU-only execution
        pin_memory=False if device == 'cpu' else True,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers
    )

In [ ]:
# Define evaluation metrics for trajectory prediction performance assessment
# These metrics are commonly used to evaluate the quality of predicted trajectories
metrics_list = ["ml_ade", "ml_fde", "nll_mean", "min_ade_5", "min_ade_10"]
# ml_ade: Most Likely Average Displacement Error - average distance between predicted and actual trajectory
# ml_fde: Most Likely Final Displacement Error - distance between predicted and actual final positions
# nll_mean: Mean Negative Log-Likelihood - measures how well the model's probability distribution fits the data
# min_ade_5: Minimum Average Displacement Error among top 5 predictions - best ADE from 5 trajectory modes
# min_ade_10: Minimum Average Displacement Error among top 10 predictions - best ADE from 10 trajectory modes

## Online (Per Agent) Plotting

In [ ]:
import matplotlib.patches as patches
from scipy import linalg

# Regular expression to extract scene name from file path
prog = re.compile("(.*)/(?P<scene_name>.*)/(.*)$")

def plot_outputs(
    eval_dataset: UnifiedDataset,
    dataset_idx: int,
    model: Trajectron,
    model_name: str,
    agent_ts: int,
    save=True,
    extra_str=None,
    subfolder="changes/",
    filetype="png"
):
    """
    Plot trajectory predictions with uncertainty visualization using ellipses.
    Shows both the most likely prediction and multiple modes with their covariances.
    """
    # Configuration parameters for visualization
    num_modes = 5  # Maximum number of prediction modes to display
    plot_every = 1  # Frequency of points to plot (1 = every point)
    plot_square_radius = 75  # Size of the plot area in meters
    pred_horizon = 30  # Number of future timesteps to predict
    
    # Get a single batch from the dataset
    batch: AgentBatch = eval_dataset.get_collate_fn(pad_format="right")([eval_dataset[dataset_idx]])
    batch_idx = 0

    # Create matplotlib figure and axis
    fig, ax = plt.subplots(dpi=150)
    
    # Plot the scene context (roads, other agents, etc.)
    # NOTE: To exactly reproduce the figure from our paper, you must edit the trajdata_vis.plot_agent_batch function
    # to accept an optional future_horizon (int) argument which truncates plotting to only the future future_horizon timesteps
    # (otherwise it can be a but hard to read the figure when full-length predictions and futures are plotted).
    # After making the above change, please add future_horizon=pred_horizon to the plot_agent_batch arguments.
    trajdata_vis.plot_agent_batch(batch, batch_idx=batch_idx, ax=ax, show=False, close=False)

    # Extract scene metadata for labeling
    scene_info_path, _, scene_ts = eval_dataset._data_index[dataset_idx]
    scene_name = prog.match(scene_info_path).group("scene_name")
    
    agent_name = batch.agent_name[0]
    agent_type_name = f"{str(AgentType(batch.agent_type[0].item()))}/{agent_name}"
    print(dataset_idx, scene_name, scene_ts, agent_type_name)

    # Generate predictions from the model
    with torch.no_grad():
        # Get the most likely prediction (deterministic)
        predictions = model.predict(batch,
                                    z_mode=True,
                                    gmm_mode=True,
                                    full_dist=False,
                                    output_dists=False)
        prediction = next(iter(predictions.values()))
        
        # Get full probability distribution over predictions
        prediction_distribution_dict, _ = model.predict(batch,
                                      z_mode=False,
                                      gmm_mode=False,
                                      full_dist=True,
                                      output_dists=True)
        pred_dist = next(iter(prediction_distribution_dict.values()))
    
    # Threshold for displaying modes (skip modes with very low probability)
    pi_threshold = 0.0
    
    # Move batch to CPU for plotting
    batch.to("cpu")
        
    # Skip if prediction shape is not as expected
    if pred_dist.mus.shape[:2] != (1, 1):
        return

    # Extract distribution parameters for visualization
    means = pred_dist.mus[batch_idx, 0].cpu().numpy()  # Mean trajectories for each mode
    covs = pred_dist.get_covariance_matrix()[batch_idx, 0].cpu().numpy()  # Covariance matrices
    pis = pred_dist.pis_cat_dist.probs[batch_idx, 0].cpu().numpy()  # Mode probabilities

    # Plot the most likely prediction trajectory
    ax.plot(
        prediction[batch_idx, 0:pred_horizon:plot_every, 0],
        prediction[batch_idx, 0:pred_horizon:plot_every, 1],
        color='#2D93AD',
        alpha=1,
        label="Most Likely"
    )
    
    # Plot each prediction mode with uncertainty ellipses
    for z_val in range(min(num_modes, means.shape[1])):
        # Get probability of this mode (same across all timesteps)
        pi = pis[0, z_val]
        
        # Skip low-probability modes
        if pi < pi_threshold:
            continue

        color = '#9F9FED'
        alpha_val = pi  # Use probability as transparency

        # Plot the mean trajectory for this mode
        ax.plot(means[0:pred_horizon:plot_every, z_val, 0], means[0:pred_horizon:plot_every, z_val, 1],
                    color=color,
                    alpha=alpha_val)

        # Plot uncertainty ellipses at each timestep
        for timestep in range(0, min(pred_horizon, means.shape[0]), plot_every):
            mean = means[timestep, z_val]
            covar = covs[timestep, z_val]

            # Compute ellipse parameters from covariance matrix
            v, w = linalg.eigh(covar)  # Eigenvalue decomposition
            v = 2. * np.sqrt(2.) * np.sqrt(v)  # Scale to 2-sigma ellipse
            u = w[0] / linalg.norm(w[0])  # Normalize eigenvector

            # Calculate rotation angle of ellipse
            angle = np.arctan2(u[1], u[0])
            angle = 180. * angle / np.pi  # convert to degrees
            
            # Create and add uncertainty ellipse
            ell = patches.Ellipse(mean, v[0], v[1], 180. + angle,
                                    color=color)
            ell.set_edgecolor(None)
            ell.set_clip_box(ax.bbox)
            ell.set_alpha(alpha_val)
            ax.add_artist(ell)
    
    # Commented out: batch evaluation metrics computation
    # batch_eval: Dict[str, torch.Tensor] = evaluation.compute_batch_statistics_pt(
    #     batch.agent_fut[..., :2],
    #     prediction_output_dict=torch.from_numpy(prediction),
    #     y_dists=pred_dist
    # )
    
    # Configure plot appearance
    ax.set_title(None)
    # ax.set_title(f"{scene_name}/t={scene_ts} {agent_type_name}")
    
    # Set plot limits with slight offset
    offset = 20
    ax.set_xlim((-plot_square_radius+offset, plot_square_radius+offset))
    ax.set_ylim((-plot_square_radius, plot_square_radius))
    
    # Remove axes and labels for clean visualization
    ax.axis('off')
    
    # Add legend outside the plot area
    ax.legend(bbox_to_anchor=(1.04, 0.5), loc="best", borderaxespad=0, frameon=False)
    
    # Save figure if requested
    if save:
        fname = f"figures/{subfolder}{model_name}_{scene_name}_{agent_name}_t{agent_ts}"
        if extra_str:
            fname += "_" + extra_str
        fig.savefig(fname + f".{filetype}", bbox_inches="tight")
        
        plt.close(fig)

In [ ]:
import time

def per_agent_plot(
    model: Trajectron,
    model_name: str,
    batch: AgentBatch,
    agent_ts: int,
    plot=True,
):
    """
    Generate and save trajectory prediction plots for a single agent at a specific timestep.
    
    This function is called during online evaluation to visualize how the model's
    predictions evolve as more data becomes available for each agent.
    
    Args:
        model: The Trajectron model to generate predictions from
        model_name: String identifier for the model (used in saved filenames)
        batch: AgentBatch containing the current agent's data and context
        agent_ts: Current timestep for this agent (used for tracking adaptation progress)
        plot: Boolean flag to enable/disable plotting (default: True)
    """
    # Disable gradient computation for inference - saves memory and speeds up prediction
    with torch.no_grad():
        if plot:
            # Call the plotting function to visualize predictions
            # Uses the dataset index from the batch to retrieve the original data sample
            # for proper scene context and ground truth visualization
            plot_outputs(online_eval_dataset,
                         dataset_idx=batch.data_idx[0].item(),  # Extract dataset index from batch
                         model=model,
                         model_name=model_name,
                         agent_ts=agent_ts)  # Pass timestep for filename generation

In [ ]:
# Create a DataLoader for online evaluation with batch size of 1 (single agent at a time)
online_eval_dataloader = get_dataloader(online_eval_dataset, batch_size=1, shuffle=False)

# Reset the adaptive model's internal state (clears any previous adaptation history)
adaptive_trajectron.reset_adaptive_info()

# Limit evaluation to first 10 samples for demonstration/testing purposes
N_SAMPLES = 10

# Create progress bar to track evaluation progress across all data samples
outer_pbar = tqdm(
    online_eval_dataloader,
    total=min(N_SAMPLES, len(online_eval_dataloader)),
    desc=f'Adaptive Eval PH={prediction_sec}',  # Show prediction horizon in description
    position=0,
)

# Flag to control whether to generate plots at each timestep
plot_per_step = True

# Variables to track current agent being processed and their timestep count
curr_agent: str = None  # Name of currently processed agent
agent_ts: int = 0       # Timestep counter for current agent
online_batch: AgentBatch  # Current batch being processed
batchlist=[]  # Store all processed batches for later analysis

# Track execution time for performance monitoring
time_list = []

# Main online evaluation loop - processes agents sequentially as data arrives
for data_sample, online_batch in enumerate(outer_pbar):
    # Start timing this iteration for performance measurement
    start_time = time.time()

    # Stop processing after reaching the sample limit
    if data_sample >= N_SAMPLES:
        outer_pbar.close()
        break
            
    # Check if we've switched to a new agent (agents are processed sequentially)
    if online_batch.agent_name[0] != curr_agent:
        # Reset adaptive parameters (K_n, L_n) when starting a new agent
        # This simulates starting fresh adaptation for each new agent
        adaptive_trajectron.reset_adaptive_info()
        
        # Commented out: Reset finetune baseline models to their initial state
        # This would be used if comparing against finetuning baselines
        # finetune_trajectron, _ = load_model(
        #     base_model, device, epoch=base_checkpoint,
        #     custom_hyperparams={"trajdata_cache_dir": "/home/bivanovic/.unified_data_cache",
        #                         "single_mode_multi_sample": False}
        # )
        # k0_finetune_trajectron, _ = load_model(
        #     k0_model, device, epoch=k0_checkpoint,
        #     custom_hyperparams={"trajdata_cache_dir": "/home/bivanovic/.unified_data_cache",
        #                         "single_mode_multi_sample": False}
        # )

        # Update current agent tracking variables
        curr_agent = online_batch.agent_name[0]
        agent_ts: int = 0  # Reset timestep counter for new agent
        
    # Disable gradient computation for inference (saves memory and computation)
    with torch.no_grad():
        # Perform adaptive prediction - this updates the model's internal parameters
        # based on the current observation using Bayesian last layer adaptation
        adaptive_trajectron.adaptive_predict(
            online_batch,
            update_mode=UpdateMode.ITERATIVE  # Update adaptively with each new observation
        )
    
    # Commented out: Update finetune baseline models with current data
    # finetune_update(finetune_trajectron, online_batch)
    # finetune_last_layer_update(k0_finetune_trajectron, online_batch)
    
    # Store current batch for later analysis or debugging
    batchlist.append(online_batch)
    
    # Generate visualization plot at regular intervals (every timestep in this case)
    if agent_ts % 1 == 0:
        per_agent_plot(adaptive_trajectron, "Ours", online_batch, agent_ts, plot=plot_per_step)
            
    # Increment timestep counter (tracks number of observations for current agent)
    agent_ts += 1
    
    # Print progress information including timing
    print(f"Agent: {curr_agent}, Agent TS: {agent_ts}, Data Sample: {data_sample + 1}/{N_SAMPLES}, Time: {time.time() - start_time:.2f}s")
    
    # Store execution time for this iteration for performance analysis
    time_list.append(time.time() - start_time)

Adaptive Eval PH=6.0:   0%|          | 0/10 [00:00<?, ?it/s]

0 scene-0103 1 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 1, Data Sample: 1/10, Time: 1.40s
1 scene-0103 2 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 2, Data Sample: 2/10, Time: 1.35s
2 scene-0103 3 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 3, Data Sample: 3/10, Time: 1.39s
3 scene-0103 4 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 4, Data Sample: 4/10, Time: 1.46s
4 scene-0103 5 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 5, Data Sample: 5/10, Time: 1.56s
5 scene-0103 6 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 6, Data Sample: 6/10, Time: 1.71s
6 scene-0103 7 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 7, Data Sample: 7/10, Time: 1.64s
7 scene-0103 8 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 8, Data Sample: 8/10, Time: 1.58s
8 scene-0103 9 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 9, Data Sample: 9/10, Time: 1.56s
9 scene-0103 10 AgentType.VEHICLE/ego
Agent: ego, Agent TS: 10, Data Sample: 10/10, Time: 1.53s


In [ ]:
# Initialize counter for tracking batch element number
i = 0

# Iterate through pairs of attributes from two different batches (timesteps 0 and 4)
# zip() pairs up corresponding attributes from both batches for comparison
# vars() converts the batch objects to dictionaries of their attributes
for (k0, v0), (k4, v4) in zip(vars(batchlist[0]).items(), vars(batchlist[4]).items()):
    print(f'Element number {i}')
    
    # Print attribute name and value from the first batch (timestep 0)
    print(f'ts0 {k0} \n {v0}')
    print("*" * 20)  # Visual separator with asterisks
    
    # Print attribute name and value from the fifth batch (timestep 4) 
    print(f'ts4 {k4} \n {v4}')
    print("-" * 20)  # Visual separator with dashes
    
    # Increment counter to track which batch attribute we're examining
    i += 1


Element number 0
ts0 data_idx 
 tensor([0], dtype=torch.int32)
********************
ts4 data_idx 
 tensor([4], dtype=torch.int32)
--------------------
Element number 1
ts0 scene_ts 
 tensor([1], device='cuda:0', dtype=torch.int32)
********************
ts4 scene_ts 
 tensor([5], device='cuda:0', dtype=torch.int32)
--------------------
Element number 2
ts0 dt 
 tensor([0.5000], device='cuda:0')
********************
ts4 dt 
 tensor([0.5000], device='cuda:0')
--------------------
Element number 3
ts0 agent_name 
 ['ego']
********************
ts4 agent_name 
 ['ego']
--------------------
Element number 4
ts0 agent_type 
 tensor([1], dtype=torch.int32)
********************
ts4 agent_type 
 tensor([1], dtype=torch.int32)
--------------------
Element number 5
ts0 curr_agent_state 
 tensor([[ 6.0357e+02,  1.6455e+03,  7.4377e+00, -4.2095e+00,  0.0000e+00,
          0.0000e+00, -5.1834e-01]], device='cuda:0')
********************
ts4 curr_agent_state 
 tensor([[ 6.1768e+02,  1.6363e+03,  6.9671e

In [30]:
len(online_eval_dataloader)

1436

In [ ]:
# Iterate through each element of the current agent's state vector
# curr_agent_state[0] contains the state information for the first (and only) agent in this batch
# The state typically includes position (x, y), velocity (vx, vy), heading, and other kinematic properties
for i in online_batch.curr_agent_state[0]:
    print(i)  # Print each state component (e.g., x-coordinate, y-coordinate, velocity components, etc.)

StateTensorXYXdYdXddYddH(637.8889)
StateTensorXYXdYdXddYddH(1619.7749)
StateTensorXYXdYdXddYddH(6.0148)
StateTensorXYXdYdXddYddH(-5.0871)
StateTensorXYXdYdXddYddH(-1.3908)
StateTensorXYXdYdXddYddH(1.2708)
StateTensorXYXdYdXddYddH(-0.6907)


In [32]:
0%1

0